In [42]:
import pandas as pd
import matplotlib.pyplot

payment_df = pd.read_csv('./data/payment.csv')
loan_df = pd.read_csv('./data/loan.csv')

# Loan Dataset

In [43]:
loan_df.head(5)

,loanId,anon_ssn,payFrequency,apr,applicationDate,originated,originatedDate,nPaidOff,approved,isFunded,loanStatus,loanAmount,originallyScheduledPaymentAmount,state,leadType,leadCost,fpStatus,clarityFraudId,hasCF
0,LL-I-07399092,beff4989be82aab4a5b47679216942fd,B,360.0,2016-02-23T17:29:01.940000,False,NaN,0.0,False,0,Withdrawn Application,500.0,978.27,IL,bvMandatory,6,NaN,5669ef78e4b0c9d3936440e6,1
1,LL-I-06644937,464f5d9ae4fa09ece4048d949191865c,B,199.0,2016-01-19T22:07:36.778000,True,2016-01-20T15:49:18.846000,0.0,True,1,Paid Off Loan,3000.0,6395.19,CA,prescreen,0,Checked,569eb3a3e4b096699f685d64,1
2,LL-I-10707532,3c174ae9e2505a5f9ddbff9843281845,B,590.0,2016-08-01T13:51:14.709000,False,NaN,0.0,False,0,Withdrawn Application,400.0,1199.45,MO,bvMandatory,3,NaN,579eab11e4b0d0502870ef2f,1
3,LL-I-02272596,9be6f443bb97db7e95fa0c281d34da91,B,360.0,2015-08-06T23:58:08.880000,False,NaN,0.0,False,0,Withdrawn Application,500.0,1074.05,IL,bvMandatory,3,NaN,555b1e95e4b0f6f11b267c18,1
4,LL-I-09542882,63b5494f60b5c19c827c7b068443752c,B,590.0,2016-06-05T22:31:34.304000,False,NaN,0.0,False,0,Rejected,350.0,814.37,NV,bvMandatory,3,NaN,5754a91be4b0c6a2bf424772,1


In [44]:
loan_df.isnull().sum()

loanId                                 256
anon_ssn                                 0
payFrequency                          1273
apr                                   3922
applicationDate                          0
originated                               0
originatedDate                      531638
nPaidOff                                24
approved                                 0
isFunded                                 0
loanStatus                             391
loanAmount                            2250
originallyScheduledPaymentAmount         0
state                                  132
leadType                                 0
leadCost                                 0
fpStatus                            525959
clarityFraudId                      219989
hasCF                                    0
dtype: int64

### Observations
- Delete 'loanId'
- See if it is possible to use 'anon_ssn'
- Extract year-data from 'applicationDate'
- Need to deal with outliers

### Null Values
- Delete 'originatedDate', 'fpStatus', 'clarityFraudId'

In [45]:
loan_df["payFrequency"].value_counts() # Replace with B

payFrequency
B    316654
W    137188
M     59092
I     33787
S     29688
Name: count, dtype: int64

In [46]:
loan_df["apr"].value_counts().iloc[:5] # Replace with 590.0

apr
590.0    224447
360.0     65111
645.0     59504
601.0     40949
680.0     39431
Name: count, dtype: int64

In [47]:
loan_df["nPaidOff"].value_counts().iloc[:5] # Replace with 0.0

nPaidOff
0.0    564599
1.0      8596
2.0      2602
3.0       918
4.0       427
Name: count, dtype: int64

In [48]:
loan_df["loanAmount"].value_counts().iloc[:5] # Replace with { (500 + 400 + 350) / 3 }

loanAmount
500.0    160547
400.0    119254
350.0     83247
300.0     62860
600.0     54640
Name: count, dtype: int64

In [49]:
loan_df["state"].value_counts().iloc[:5] # Replace with OH

state
OH    90496
IL    66710
TX    49729
MO    49249
WI    40349
Name: count, dtype: int64

In [60]:
loan_df['anon_ssn'].value_counts()[:10] # Use by encoding because history of the applicant is important

anon_ssn
c8bb49de1f8ff99d2ecddfb7037dc66e    35
0b87684b60c8b8f5d0bd40eb5811cd50    20
f971898a3c70e4c3c316cb4bc48b6e2e    18
6f0e71bc16f634fafcf5337cecdfb25e    16
64cd38e9cd1e06ed480b5e607494ce57    15
161f764c0bff5b9605d9283c21ab6926    15
65caf69fa6779296fe50b1433fadd811    14
fd0e0e5ca3b308eba0b11a6b06ca658b    14
6946e99d05989d677799e8ef321262be    14
f76c6bd798b03dbf1c697902c0a6a687    14
Name: count, dtype: int64

### Understanding target variable

In [50]:
loan_df['loanStatus'].value_counts()

loanStatus
Withdrawn Application          450984
Rejected                        85070
Paid Off Loan                   11427
External Collection             11334
New Loan                         8112
Internal Collection              5567
Returned Item                    1182
CSR Voided New Loan              1026
Settlement Paid Off               708
Credit Return Void                704
Customer Voided New Loan          504
Settled Bankruptcy                326
Pending Paid Off                  169
Charged Off Paid Off              160
Pending Application Fee             5
Pending Rescind                     4
Pending Application                 4
Voided New Loan                     2
Charged Off                         1
Customver Voided New Loan           1
Settlement Pending Paid Off         1
Name: count, dtype: int64

Looking at the provided loanStatus values, we can identify certain categories that are indicative of high risk:

- Rejected: Loans that were rejected at the application stage might signify that the applicant was deemed to be at risk or not creditworthy.
- External Collection: Loans that have entered external collections often indicate a significant risk of default.
- Internal Collection: Similarly, loans in internal collections suggest potential payment difficulties.
- Charged Off: This typically means the lender has given up on collecting the debt, indicating a high-risk scenario.
- Settled Bankruptcy: Loans that have been settled due to bankruptcy also suggest a higher level of risk.

The rest of the values are considered to be low-risk.

In [51]:
high_risk = ['Rejected' , 'External Collection' , 'Internal Collection' , 'Charged Off' , 'Settled Bankruptcy']
binaryRiskStatus = ['High' if x in high_risk else 'Low' for x in loan_df['loanStatus']]
loan_df['binaryRiskStatus'] = binaryRiskStatus
loan_df['binaryRiskStatus'].value_counts()

binaryRiskStatus
Low     475384
High    102298
Name: count, dtype: int64

- Class imbalance issue
- Need to indicate during LightBGM training for internal handling

# Payment Dataset

In [52]:
payment_df.head(5)

,loanId,installmentIndex,isCollection,paymentDate,principal,fees,paymentAmount,paymentStatus,paymentReturnCode
0,LL-I-00000021,1,False,2014-12-19T05:00:00,22.33,147.28,169.61,Checked,NaN
1,LL-I-00000021,2,False,2015-01-02T05:00:00,26.44,143.17,169.61,Checked,NaN
2,LL-I-00000021,3,False,2015-01-16T05:00:00,31.30,138.31,169.61,Checked,NaN
3,LL-I-00000021,4,False,2015-01-30T05:00:00,37.07,132.54,169.61,Checked,NaN
4,LL-I-00000021,5,False,2015-02-13T05:00:00,43.89,125.72,169.61,Checked,NaN


#### How to use the payment dataset?
- Group by loanId and use feature engineering to extract valuable data.
- Attach to the loan dataset on loanId.

In [53]:
payment_df.shape

(689364, 9)

In [54]:
len(payment_df['loanId'].unique())

39952

- Around 700K rows in payment dataset.
- Only 40k unique IDs in payment dataset.
- Features engineered using payment dataset will be applied to only 40k data points.
- Dataset deemed not useful.